<a href="https://colab.research.google.com/github/marcelofschiavo/ds-cookbook/blob/main/05_ML_Classificacao_(Supervisionado).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 05. Modelagem Preditiva (Classificação)

Nos notebooks anteriores, nós *entendemos* o passado (com EDA e Estatística).
Agora, vamos *prever* o futuro.

**Objetivo:** Criar um modelo de Machine Learning que aprenda com os dados de quem *saiu* e *ficou* para prever, com base em um funcionário novo (ou existente), qual a **probabilidade** dele pedir demissão.

**Tipo de Problema:** Classificação Binária (prever uma categoria: "Sim" ou "Não").

In [ ]:
"""
## Setup: Carregar Bibliotecas e Dados

Nesta célula, importamos as bibliotecas que usaremos (Pandas e Scikit-learn)
e carregamos nosso conjunto de dados limpo do Notebook 01.
"""
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import warnings

# Ignorar avisos futuros (apenas para limpar o output)
warnings.filterwarnings('ignore')

# --- Carregue seus dados aqui ---
# (Substitua 'dados_empresa_limpos.csv' pelo nome do seu arquivo)
try:
    df = pd.read_csv('dados_empresa_limpos.csv')
    print("DataFrame 'df' carregado com sucesso!")
    print(f"Total de {df.shape[0]} linhas e {df.shape[1]} colunas.")

except FileNotFoundError:
    print("------------------------------------------------------------------")
    print(">>> ERRO: Arquivo 'dados_empresa_limpos.csv' não encontrado. <<<")
    print("... Criando um DataFrame de EXEMPLO para o resto do notebook não quebrar.")
    print("------------------------------------------------------------------")

    # Criar um 'df' de exemplo para classificação
    data = {
        'salario': np.random.randint(3000, 15000, 500),
        'satisfacao': np.random.rand(500).round(2),
        'tempo_empresa': np.random.randint(1, 10, 500),
        'departamento': np.random.choice(['TI', 'Vendas', 'RH'], 500),
        'pediu_demissao': np.random.choice([0, 1], 500, p=[0.85, 0.15]) # 15% de turnover
    }
    df = pd.DataFrame(data)
    print("DataFrame de exemplo carregado.")

print("\nAmostra dos dados:")
print(df.head())

## Conceito 1: Preparação (Separar X/y e One-Hot Encoding)

🧠 **Intuição:**
"Modelos de ML são ótimos em matemática, mas péssimos em ler texto. Não podemos dar a ele a coluna 'departamento' com 'TI' ou 'Vendas'. Precisamos 'traduzir':
1.  **Separar:** Dizemos ao modelo o que ele precisa prever (o 'y': `pediu_demissao`) e quais 'pistas' ele pode usar (o 'X': todas as outras colunas).
2.  **Traduzir:** Transformamos 'departamento' em colunas numéricas, como 'departamento_TI' (1 se sim, 0 se não) e 'departamento_Vendas' (1 se sim, 0 se não)."

🎓 **Definição Técnica:**
Realizamos duas operações:
1.  **Separação de Features e Target:** `y` é o vetor-alvo (variável dependente, `pediu_demissao`) e `X` é a matriz de features (variáveis independentes, preditores).
2.  **One-Hot Encoding:** Usamos `pd.get_dummies()` para converter variáveis categóricas nominais (como `departamento`) em N colunas binárias, onde N é o número de categorias únicas. Isso evita que o modelo assuma uma ordem incorreta (ex: que TI < Vendas < RH).

🍳 **Receita:**

In [ ]:
# 1. Definir X (features) e y (target)
#    (Assumindo que 'pediu_demissao' é 1 para Sim e 0 para Não)
y = df['pediu_demissao']
X = df.drop('pediu_demissao', axis=1) # Usar todas as outras colunas como features

# 2. One-Hot Encoding (Converter categorias em números)
#    O 'drop_first=True' evita redundância e multicolinearidade
X_encoded = pd.get_dummies(X, drop_first=True, dtype=int)

print("--- Target (y) ---")
print(y.head())
print("\n--- Features (X_encoded) ---")
print(X_encoded.head())

📊 **Resultado:**
"Agora temos 'y' (uma Série só com 0s e 1s) e 'X_encoded' (um DataFrame só com números, pronto para o modelo)."

## Conceito 2: Divisão de Treino e Teste (train_test_split)

🧠 **Intuição:**
"Nunca podemos testar um aluno usando a mesma prova que ele usou para estudar. Ele iria 'decorar' as respostas, não 'aprender' o conceito.
Com ML é igual. Nós dividimos nosso 'baralho' de dados:
* **70% para Treino:** As 'aulas' e 'exercícios' que o modelo usa para aprender os padrões.
* **30% para Teste:** A 'prova final'. Dados que o modelo *nunca viu*, usados para ver se ele realmente aprendeu a generalizar."

🎓 **Definição Técnica:**
Particionamos os dados (X e y) em conjuntos de treino e teste. Isso é vital para avaliar a capacidade de **generalização** do modelo em dados "novos" (out-of-sample).
* `test_size=0.3`: Reserva 30% dos dados para o teste.
* `random_state=42`: Garante a **reprodutibilidade**. A divisão será sempre a mesma, não importa quantas vezes rodarmos.
* `stratify=y`: **CRUCIAL** em classificação desbalanceada (como turnover). Garante que a proporção de 'Sim' e 'Não' (ex: 15% de 'Sim') seja a mesma tanto no conjunto de treino quanto no de teste.

🍳 **Receita:**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y,
    test_size=0.30,
    random_state=42,
    stratify=y  # Essencial para garantir proporções iguais de 'y' no treino/teste
)
print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test:  {X_test.shape}")
print("-" * 20)
print(f"Proporção de 'Sair' no Treino: {y_train.mean():.2%}")
print(f"Proporção de 'Sair' no Teste:  {y_test.mean():.2%}")

📊 **Resultado:**
"Nossos dados estão divididos. O modelo treinará com as amostras de 'treino' e será avaliado com as amostras de 'teste'."

## Conceito 3: Feature Scaling (StandardScaler) - (Opcional, mas recomendado)

🧠 **Intuição:**
"Alguns modelos (como a Regressão Logística) são 'enganados' por escalas diferentes. A coluna 'salario' (Ex: 3000 a 15000) parece *milhares* de vezes mais importante que 'satisfacao' (Ex: 0 a 1), mesmo que não seja.
O 'Scaler' age como um 'equalizador': ele coloca todas as features na mesma régua (média 0, desvio padrão 1), garantindo uma comparação justa."

🎓 **Definição Técnica:**
A Padronização (Standardization) transforma os dados para que tenham média 0 e desvio padrão 1 (Z-score). É crucial para algoritmos sensíveis à escala (Regressão Logística, SVMs, KNN, Redes Neurais).
**IMPORTANTE (Evitar Data Leakage):** O scaler deve ser treinado (`.fit()`) **APENAS** nos dados de treino (`X_train`) e depois usado para transformar (`.transform()`) ambos (`X_train` e `X_test`).

🍳 **Receita:**

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1. Criar o Scaler
scaler = StandardScaler()

# 2. Treinar (fit) o scaler APENAS no X_train
scaler.fit(X_train)

# 3. Aplicar (transform) o scaler em ambos os conjuntos
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Média de X_train (antes): {X_train['salario'].mean():.2f}")
print(f"Média de X_train_scaled (depois): {X_train_scaled[:, 0].mean():.2f}") # Pega a primeira coluna
print(f"Desvio Padrão de X_train_scaled (depois): {X_train_scaled[:, 0].std():.2f}")

📊 **Resultado:**
"Os dados de treino e teste agora estão escalados. Note como a média de 'X_train_scaled' é ~0 e o desvio padrão é ~1."

## Conceito 4: Modelo 1 - Regressão Logística (O Baseline)

🧠 **Intuição:**
"É o modelo 'baseline', o mais simples e rápido. Ele acha a melhor 'linha' (ou curva sigmóide) que separa os 'Sim' dos 'Não'. É ótimo para ter um resultado rápido e para ser um ponto de partida (se um modelo complexo não for muito melhor que esse, talvez não valha a pena usá-lo)."

🎓 **Definição Técnica:**
A Regressão Logística é um modelo linear generalizado usado para classificação binária. Ele modela a *probabilidade* de uma classe (P(y=1)) usando a função logística (sigmóide).
* `class_weight='balanced'`: Ajusta os pesos do modelo para dar mais importância à classe minoritária (no nosso caso, 'pediu_demissao'=1), o que é vital para dados desbalanceados.

🍳 **Receita:**

In [ ]:
from sklearn.linear_model import LogisticRegression

# 1. Criar o modelo
#    Usamos 'class_weight='balanced'' para o modelo dar mais importância
#    aos casos de 'Sim' (turnover), que geralmente são minoria.
log_reg = LogisticRegression(random_state=42, class_weight='balanced')

# 2. Treinar o modelo (usando dados ESCALADOS)
log_reg.fit(X_train_scaled, y_train)

# 3. Fazer previsões (nos dados de teste ESCALADOS)
y_pred_log_reg = log_reg.predict(X_test_scaled)

print("Previsões da Regressão Logística (primeiros 10):")
print(y_pred_log_reg[:10])

📊 **Resultado:**
"Modelo treinado. As previsões estão na variável 'y_pred_log_reg'. Vamos avaliá-las no próximo passo."

## Conceito 5: Modelo 2 - Random Forest (O "Cavalo de Batalha")

🧠 **Intuição:**
"É o 'cavalo de batalha' do ML. Em vez de uma pessoa tentando decidir, ele pergunta a 100 'especialistas' (Árvores de Decisão) e faz uma *votação* entre elas. Cada 'especialista' vê o problema de um ângulo um pouco diferente. O resultado da 'votação' é muito mais robusto e preciso.
**Bônus:** Modelos de árvore não se importam com a escala dos dados (não precisam do Scaler)."

🎓 **Definição Técnica:**
O Random Forest é um método *ensemble* de *bagging*. Ele constrói múltiplas árvores de decisão em tempo de treino, cada uma em uma sub-amostra aleatória dos dados (bootstrap). A previsão final é a *moda* (votação) das previsões de todas as árvores. É robusto a overfitting e captura relações não-lineares complexas.

🍳 **Receita:**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Criar o modelo
rf_model = RandomForestClassifier(
    n_estimators=100,  # Quantidade de árvores na "floresta"
    random_state=42,
    class_weight='balanced',
    n_jobs=-1          # Usar todos os processadores
)

# 2. Treinar o modelo (usando dados ORIGINAIS, não escalados)
rf_model.fit(X_train, y_train)

# 3. Fazer previsões (nos dados de teste ORIGINAIS)
y_pred_rf = rf_model.predict(X_test)
print("Previsões do Random Forest (primeiros 10):")
print(y_pred_rf[:10])

📊 **Resultado:**
"Modelo treinado. As previsões estão na variável 'y_pred_rf'. Agora vamos comparar os dois modelos."

## Conceito 6: Avaliação (Classification Report & Confusion Matrix)

🧠 **Intuição:**
"Em turnover, 'Acurácia' (acertos totais) é inútil. Um modelo que chuta 'Fica' para todos acerta 90% das vezes (se 10% saem), mas não serve para nada. Precisamos focar nas métricas certas:
* **Recall (para classe 1 'Sair'):** 'De todas as pessoas que *realmente* vão sair, quantas o nosso modelo conseguiu *pegar*?' (Esta é a métrica-chave para o RH!).
* **Precisão (para classe 1 'Sair'):** 'De todos que o modelo *acusou* que iam sair, quantos realmente saíram?' (Importante para não gastar recursos de retenção com quem ia ficar)."

A **Matriz de Confusão** é o 'raio-X' dos erros. O pior erro é o **Falso Negativo**: a pessoa *vai sair* (Real=1) e o modelo previu que ela ia *ficar* (Previsto=0)."

🎓 **Definição Técnica:**
* **`classification_report`**: Fornece um resumo das principais métricas (Precisão, Recall, F1-Score) por classe. O F1-Score é a média harmônica de Precisão e Recall, útil como métrica única.
* **`confusion_matrix`**: Uma tabela que resume o desempenho:
    * **TN (Topo Esquerdo):** Previu '0', Real '0'.
    * **FP (Topo Direita):** Previu '1', Real '0'. (Erro Tipo I)
    * **FN (Baixo Esquerda):** Previu '0', Real '1'. (Erro Tipo II - O mais crítico!)
    * **TP (Baixo Direita):** Previu '1', Real '1'.

🍳 **Receita:**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Vamos avaliar o Random Forest (y_pred_rf), que geralmente é melhor.
print("--- Classification Report (Random Forest) ---")
# 'target_names' deixa o relatório mais legível
print(classification_report(y_test, y_pred_rf, target_names=['Ficou (0)', 'Saiu (1)']))


# --- Matriz de Confusão ---
print("\n--- Matriz de Confusão (Random Forest) ---")
cm = confusion_matrix(y_test, y_pred_rf)

# Plotar a matriz
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Ficou (0)', 'Saiu (1)'])
disp.plot(cmap='Blues', values_format='d') # 'd' = formato decimal
plt.title('Matriz de Confusão')
plt.show()

📊 **Resultado:**
"No Classification Report, olhe para a linha 'Saiu (1)'. Um **Recall de 0.70** significaria que estamos 'pegando' 70% das pessoas que realmente vão pedir demissão.
Na Matriz de Confusão, o número no canto **inferior esquerdo** são os Falsos Negativos (as pessoas que perdemos). Nosso objetivo é minimizar esse número."

## Conceito 7: Interpretação (Feature Importance)

🧠 **Intuição:**
"OK, o modelo do Random Forest é uma 'caixa-preta' que acerta bastante. Mas *por quê*? Ele não pode simplesmente dizer 'prevejo que ele vai sair' sem uma explicação.
A 'Importância das Features' nos diz quais 'pistas' (colunas) o modelo mais usou para tomar suas decisões. Ele nos diz o 'O Quê' do porquê (Ex: 'Salário' é o mais importante), dando um foco de ação para o RH."

🎓 **Definição Técnica:**
Em modelos de árvore (como o Random Forest), podemos extrair o `feature_importances_`. Esta é uma medida (geralmente *Gini importance*) que calcula o quanto cada feature contribuiu, em média, para a redução da impureza (ou aumento da 'pureza' dos nós) ao longo de todas asárvores da floresta.

🍳 **Receita:**

In [ ]:
import seaborn as sns

# 1. Pegar as importâncias do modelo treinado
importances = rf_model.feature_importances_

# 2. Criar um DataFrame para visualização
#    Usamos X_encoded.columns para pegar os nomes das features
feature_importance_df = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 3. Plotar as 10 mais importantes
plt.figure(figsize=(10, 6))
sns.barplot(
    x='Importance',
    y='Feature',
    data=feature_importance_df.head(10),
    palette='viridis'
)
plt.title('Top 10 Features Mais Importantes (Random Forest)')
plt.xlabel('Importância')
plt.ylabel('Feature')
plt.show()

📊 **Resultado:**
"O gráfico confirma visualmente quais são os principais *drivers* do turnover. Se 'satisfacao', 'salario' e 'tempo_empresa' estão no topo, isso valida nossas suspeitas da EDA e da Estatística, e dá ao RH um foco claro de atuação para políticas de retenção."